# Redefining Geographies for Custom Geographies in Replcia

**What I have:** already defined searate census tracts in replica's saved selections, one for each station. 
</br>
**What I need:** combined geographies for all tracts from the already defined stations except for the station listed as origin

In [3]:
import geopandas as gpd
import pandas as pd
from siuba import *
import numpy as np

from calitp_data_analysis.sql import to_snakecase

from calitp_data_analysis import get_fs
import gcsfs as fs
fs = get_fs()

In [4]:
gcs_path = "gs://calitp-analytics-data/data-analyses/big_data/lossan_stations/"

In [10]:
station_list = ["simi-valley", "burbank-airport", "oxnard",  "camarillo", "moorpark",  "chatsworth",
                "northridge", "van-nuys", "burbank", "glendale", "untion-station", "commerce", "norwalk", "buena-park",
                "fullerton", "anaheim",  "orange", "santa-ana", "tustin",
                "irvine", "laguna-niguel", "san-juan-capistrano", "san-clemente", "san-clemente-pier",
                "oceanside", "solana-beach", "san-diego", "ot-san-diego"]

In [6]:
with get_fs().open(f"{gcs_path}saved-selection-geo-lossan-anaheim_us-tract-2020.zip") as f:
        test = to_snakecase(gpd.read_file(f))

In [9]:
# test.explore()